# Text Processing

This notebook imports scraped data, splits text column to words, extracts and normalizes tokens. Next, tokens are converted to embeddings.

In [10]:
### Setup

# pip install spacy
# python -m spacy download de_core_news_sm  - for small model (13 MB)
# python -m spacy download de_core_news_md  - for medium model (42 MB)
# source - https://spacy.io/models/de


import spacy
from spacy.lang.de.examples import sentences 


# Example from documentation
nlp = spacy.load("de_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Die ganze Stadt ist ein Startup: Shenzhen ist das Silicon Valley für Hardware-Firmen
Die DET nk
ganze ADJ nk
Stadt NOUN sb
ist AUX ROOT
ein DET nk
Startup NOUN pd
: PUNCT punct
Shenzhen NOUN sb
ist AUX cj
das DET nk
Silicon PROPN pnc
Valley PROPN sb
für ADP mnr
Hardware-Firmen NOUN nk


In [11]:
import pandas as pd

standard_data = pd.read_csv('./data/derstandard_frontpage_data.csv')[0:1000]
standard_data

,title,subtitle,link,datetime,kicker,n_posts,storylabels
0,Liverpool nach großem Zittern im Ligacup-Halbf...,Minamino rettete die Reds in der Nachspielzeit...,https://www.derstandard.at/story/2000132122809...,2021-12-22T23:47,Premier League,7,NaN
1,USA lassen Pfizers Covid-Tablette Paxlovid für...,Am Dienstag wurden in Österreich 2.269 Neuinfe...,https://www.derstandard.at/jetzt/livebericht/2...,2021-12-22T23:39,Omikron-Welle,22127,NachleseLivebericht
2,Real Madrid baut Vorsprung in LaLiga aus,Madrilenen nach 2:1 in Bilbao acht Punkte vor ...,https://www.derstandard.at/story/2000132122694...,2021-12-22T23:38,Primera Division,40,NaN
3,Paris Saint-Germain wendet zweite Liga-Niederl...,Icardi besorgteAusgleichstreffergegen Grbic-Te...,https://www.derstandard.at/story/2000132122616...,2021-12-22T23:30,Fußball,3,NaN
4,OSZE verkündet Einigung auf Waffenstillstand i...,"DieKonfliktparteiensollen zugestimmt haben, da...",https://www.derstandard.at/story/2000132122215...,2021-12-22T22:33,Weihnachtsfrieden,44,NaN
...,...,...,...,...,...,...,...
995,Mit Desmond Tutu wurde eine Epoche zu Grabe ge...,Am Wochenende fanden die Gedenkfeiern ihren Hö...,https://www.derstandard.at/story/2000132266302...,2022-01-02T14:53,Südafrika,17,NaN
996,Ermittlungen eingeleitet: Rechtsextreme kündig...,Auf einer Feier soll sich Gottfried Küssel mit...,https://www.derstandard.at/story/2000132266047...,2022-01-02T14:29,Laut Puls 24,494,NaN
997,Bewegung: Der richtige Sport für Ihren Blutdruck,Sport und Bewegung können einen zu hohen Blutd...,https://www.derstandard.at/story/2000132200522...,2022-01-02T14:00,Gesunder Lifestyle,448,NaN
998,Sozialwohnungen in Singapur: Mit Kreativität g...,Der soziale Wohnbau in Singapur ist oft eintön...,https://www.derstandard.at/story/2000132049781...,2022-01-02T14:00,Von Yacht bis Sake-Bar,86,Ansichtssache


## Part 1: Tokenizing text columns

In [12]:
standard_data['title_tokens'] = standard_data['title']
standard_data['subtitle_tokens'] = standard_data['subtitle']

In [13]:
for index, row in standard_data.iterrows():
    
    text = row['title_tokens']
    doc = nlp(text)
    token_list = [token for token in doc]
    standard_data.at[index, 'title_tokens'] = token_list 

    try:

        text = row['subtitle_tokens']
        doc = nlp(text)
        token_list = [token for token in doc]

    except:
        token_list = []

    standard_data.at[index, 'subtitle_tokens'] = token_list



In [14]:
standard_data

,title,subtitle,link,datetime,kicker,n_posts,storylabels,title_tokens,subtitle_tokens
0,Liverpool nach großem Zittern im Ligacup-Halbf...,Minamino rettete die Reds in der Nachspielzeit...,https://www.derstandard.at/story/2000132122809...,2021-12-22T23:47,Premier League,7,NaN,"[Liverpool, nach, großem, Zittern, im, Ligacup...","[Minamino, rettete, die, Reds, in, der, Nachsp..."
1,USA lassen Pfizers Covid-Tablette Paxlovid für...,Am Dienstag wurden in Österreich 2.269 Neuinfe...,https://www.derstandard.at/jetzt/livebericht/2...,2021-12-22T23:39,Omikron-Welle,22127,NachleseLivebericht,"[USA, lassen, Pfizers, Covid-Tablette, Paxlovi...","[Am, Dienstag, wurden, in, Österreich, 2.269, ..."
2,Real Madrid baut Vorsprung in LaLiga aus,Madrilenen nach 2:1 in Bilbao acht Punkte vor ...,https://www.derstandard.at/story/2000132122694...,2021-12-22T23:38,Primera Division,40,NaN,"[Real, Madrid, baut, Vorsprung, in, LaLiga, aus]","[Madrilenen, nach, 2:1, in, Bilbao, acht, Punk..."
3,Paris Saint-Germain wendet zweite Liga-Niederl...,Icardi besorgteAusgleichstreffergegen Grbic-Te...,https://www.derstandard.at/story/2000132122616...,2021-12-22T23:30,Fußball,3,NaN,"[Paris, Saint-Germain, wendet, zweite, Liga-Ni...","[Icardi, besorgteAusgleichstreffergegen, Grbic..."
4,OSZE verkündet Einigung auf Waffenstillstand i...,"DieKonfliktparteiensollen zugestimmt haben, da...",https://www.derstandard.at/story/2000132122215...,2021-12-22T22:33,Weihnachtsfrieden,44,NaN,"[OSZE, verkündet, Einigung, auf, Waffenstillst...","[DieKonfliktparteiensollen, zugestimmt, haben,..."
...,...,...,...,...,...,...,...,...,...
995,Mit Desmond Tutu wurde eine Epoche zu Grabe ge...,Am Wochenende fanden die Gedenkfeiern ihren Hö...,https://www.derstandard.at/story/2000132266302...,2022-01-02T14:53,Südafrika,17,NaN,"[Mit, Desmond, Tutu, wurde, eine, Epoche, zu, ...","[Am, Wochenende, fanden, die, Gedenkfeiern, ih..."
996,Ermittlungen eingeleitet: Rechtsextreme kündig...,Auf einer Feier soll sich Gottfried Küssel mit...,https://www.derstandard.at/story/2000132266047...,2022-01-02T14:29,Laut Puls 24,494,NaN,"[Ermittlungen, eingeleitet, :, Rechtsextreme, ...","[Auf, einer, Feier, soll, sich, Gottfried, Küs..."
997,Bewegung: Der richtige Sport für Ihren Blutdruck,Sport und Bewegung können einen zu hohen Blutd...,https://www.derstandard.at/story/2000132200522...,2022-01-02T14:00,Gesunder Lifestyle,448,NaN,"[Bewegung, :, Der, richtige, Sport, für, Ihren...","[Sport, und, Bewegung, können, einen, zu, hohe..."
998,Sozialwohnungen in Singapur: Mit Kreativität g...,Der soziale Wohnbau in Singapur ist oft eintön...,https://www.derstandard.at/story/2000132049781...,2022-01-02T14:00,Von Yacht bis Sake-Bar,86,Ansichtssache,"[Sozialwohnungen, in, Singapur, :, Mit, Kreati...","[Der, soziale, Wohnbau, in, Singapur, ist, oft..."
